### 构建卷积神经网络
- 卷积网络中的输入和层与传统神经网络有些区别，需重新设计，训练模块基本一致

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 首先读取数据
- 分别构建训练集和测试集（验证集）
- DataLoader来迭代取数据

In [2]:
# 定义超参数 
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 训练集
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# 测试集
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

### 卷积网络模块构建
- 一般卷积层，relu层，池化层可以写成一个套餐
- 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # 灰度图
                out_channels=16,            # 要得到几多少个特征图
                kernel_size=5,              # 卷积核大小
                stride=1,                   # 步长
                padding=2,                  # 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1
            ),                              # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作输出 (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # 下一个卷积模块的输入 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # 卷积之后的结果(32, 14, 14)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(2),                # 池化层输出 (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # 全连接层得到的结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

### 准确率作为评估标准

In [4]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1] 
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    return rights, len(labels) 

### 训练网络模型

In [5]:
# 实例化
net = CNN() 
#损失函数
criterion = nn.CrossEntropyLoss() 
#优化器，普通的随机梯度下降
optimizer = optim.Adam(net.parameters(), lr=0.001) # 定义优化器，

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    # 针对容器中的每一个批进行循环
    for batch_idx, (data, target) in enumerate(train_loader):  
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        right = accuracy(output, target) 
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: 
            
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))

当前epoch: 0 [0/60000 (0%)]	损失: 2.311086	训练集准确率: 1.56%	测试集正确率: 13.59%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.330137	训练集准确率: 75.19%	测试集正确率: 91.46%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.148561	训练集准确率: 83.85%	测试集正确率: 95.00%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.062015	训练集准确率: 87.58%	测试集正确率: 96.14%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.099082	训练集准确率: 89.72%	测试集正确率: 97.07%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.112609	训练集准确率: 91.09%	测试集正确率: 97.20%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.141436	训练集准确率: 92.11%	测试集正确率: 97.36%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.099590	训练集准确率: 92.88%	测试集正确率: 97.80%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.051906	训练集准确率: 93.45%	测试集正确率: 98.14%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.103150	训练集准确率: 93.89%	测试集正确率: 98.24%
当前epoch: 1 [0/60000 (0%)]	损失: 0.013676	训练集准确率: 100.00%	测试集正确率: 98.24%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.029473	训练集准确率: 98.24%	测试集正确率: 98.02%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.020634	训练集准确率: 98.31%	测试集正确率: 98.26%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.045692	训练集准确率